# AG2 Context Variables vs Event Processing

This notebook explores the key differences between **AG2 Context Variables** and **Tool/Event Processing**, clarifying how each should be used for different purposes in your MozaiksAI system.

## Key Concepts:

- **Context Variables**: Simple key-value state sharing between agents (NOT tools)
- **Tools**: Interactive functions that agents can call to perform actions
- **Events**: Asynchronous UI interactions that require user input/responses

Let's dive into each concept with practical examples!

## 1. Import Required Libraries and Setup

Let's start by importing the necessary AG2 libraries and understanding what we're working with:

In [ ]:
# Import AG2 Core Libraries
from autogen import ConversableAgent, GroupChat, GroupChatManager
from autogen.agentchat.group import ContextVariables
import asyncio
import uuid
import logging
from typing import Dict, Any, Optional
from datetime import datetime

# Setup logging for examples
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ AG2 libraries imported successfully!")
print("📚 Key imports:")
print("   - ContextVariables: For state sharing between agents")
print("   - ConversableAgent: For creating interactive agents")
print("   - GroupChat/GroupChatManager: For orchestrating conversations")

## 2. Understanding Context Variables in AG2

**Context Variables are NOT tools!** They are simple key-value pairs for sharing state between agents.

### What Context Variables ARE:
- ✅ Simple data storage accessible to all agents in a group chat
- ✅ Automatic serialization/deserialization by AG2
- ✅ Passed implicitly to agent conversations
- ✅ Perfect for storing workflow state, user preferences, API keys

### What Context Variables are NOT:
- ❌ Interactive functions that agents call
- ❌ Tools for user input/output
- ❌ Event handlers or UI components
- ❌ Complex objects or classes

In [ ]:
# Example: Creating and Using Context Variables
def create_context_variables():
    """Proper way to create context variables"""
    
    # Context variables are simple dictionaries
    context_vars = ContextVariables()
    
    # Set simple values
    context_vars.set("user_name", "John Doe")
    context_vars.set("workflow_type", "generator")
    context_vars.set("api_keys", {
        "openai": "sk-xxx",
        "anthropic": "ak-xxx"
    })
    context_vars.set("concept_overview", "Build a customer support chatbot")
    context_vars.set("project_requirements", [
        "Real-time chat interface",
        "Knowledge base integration", 
        "Multi-language support"
    ])
    
    return context_vars

# Create example context variables
example_context = create_context_variables()

print("🎯 Context Variables Created:")
print(f"   user_name: {example_context.get('user_name')}")
print(f"   workflow_type: {example_context.get('workflow_type')}")
print(f"   concept_overview: {example_context.get('concept_overview')}")
print(f"   api_keys available: {list(example_context.get('api_keys', {}).keys())}")
print(f"   requirements count: {len(example_context.get('project_requirements', []))}")

print("\n✅ Context variables are simple data storage - no function calls needed!")

## 3. Setting and Reading Context Variables in Agents

Agents can access context variables automatically through the conversation context:

In [ ]:
# Example: Agent that reads context variables
def create_context_aware_agent():
    """Agent that can read and use context variables"""
    
    agent = ConversableAgent(
        name="ContextAgent",
        system_message="""You are the ContextAgent. You can access context variables automatically.
        
        Available context variables:
        - user_name: The user's name
        - concept_overview: Description of what to build
        - project_requirements: List of requirements
        - api_keys: Available API keys
        
        When responding, reference these context variables to provide personalized responses.
        Context variables are automatically available - you don't need to call any tools to access them.
        """,
        llm_config=False,  # Simplified for example
        human_input_mode="NEVER"
    )
    
    return agent

# Create the agent
context_agent = create_context_aware_agent()

print("🤖 Created ContextAgent")
print("📝 Agent system message explains how to access context variables")
print("🔑 Key insight: Context variables are AUTOMATIC - no tool calls needed!")

print("\n💡 In your workflow:")
print("   1. Set context variables once at the start")
print("   2. Agents automatically receive them in conversations") 
print("   3. Agents can reference values directly in their responses")
print("   4. No tools, functions, or special calls required!")

## 4. Event Processing with AG2

**Events** are different from context variables - they handle asynchronous interactions and user responses.

### Event Processing Use Cases:
- ✅ Requesting user input (API keys, preferences)
- ✅ UI interactions that need responses
- ✅ Asynchronous operations that take time
- ✅ Real-time updates to the frontend

### Event vs Context Variable:
- **Context Variable**: "Store the API key for later use"
- **Event**: "Ask the user for their API key and wait for response"

In [ ]:
# Example: Event Emission and Response Handling
class MockEventSystem:
    """Simplified version of your transport system for demonstration"""
    
    def __init__(self):
        self.pending_events = {}
        self.responses = {}
    
    async def emit_ui_event(self, tool_id: str, payload: Dict[str, Any]) -> str:
        """Emit a UI event and return event ID"""
        event_id = f"{tool_id}_{str(uuid.uuid4())[:8]}"
        
        print(f"🎯 Emitting UI event: {tool_id}")
        print(f"   Event ID: {event_id}")
        print(f"   Payload: {payload}")
        
        # Store the event
        self.pending_events[event_id] = {
            "tool_id": tool_id,
            "payload": payload,
            "timestamp": datetime.utcnow()
        }
        
        # Simulate sending to frontend
        print(f"📡 Event sent to frontend for user interaction")
        
        return event_id
    
    async def wait_for_response(self, event_id: str, timeout: float = 10.0) -> Dict[str, Any]:
        """Wait for user response to an event"""
        print(f"⏳ Waiting for user response to event: {event_id}")
        
        # Simulate user response (in real system, this comes from WebSocket)
        await asyncio.sleep(0.1)  # Simulate network delay
        
        # Mock response
        mock_response = {
            "event_id": event_id,
            "status": "success",
            "data": {
                "api_key": "sk-mock-api-key-12345",
                "service": "openai"
            },
            "timestamp": datetime.utcnow().isoformat()
        }
        
        print(f"✅ Received user response: {mock_response['status']}")
        print(f"   Data: {mock_response['data']}")
        
        return mock_response

# Test the event system
event_system = MockEventSystem()

async def test_event_flow():
    # Emit an API key request event
    event_id = await event_system.emit_ui_event(
        tool_id="api_key_input",
        payload={
            "service": "openai",
            "label": "OpenAI API Key",
            "placeholder": "Enter your OpenAI API key..."
        }
    )
    
    # Wait for response
    response = await event_system.wait_for_response(event_id)
    
    return response

# Run the test
response = await test_event_flow()
print(f"\n🎉 Event flow completed! Got API key: {response['data']['api_key'][:10]}...")

## 5. Tool Calls vs Direct Event Handling

Let's compare different approaches for handling user interactions:

In [ ]:
# Comparison: Tool-based vs Event-based approaches

print("🔧 APPROACH 1: Tool-Based (What you currently have)")
print("=====================================")
print("✅ Pros:")
print("   - Agent explicitly calls request_api_key() tool")
print("   - Clear function interface with parameters")
print("   - Tool is registered with agent via @register_for_execution")
print("   - Follows standard AG2 tool patterns")
print()
print("❌ Cons:")
print("   - More complex setup (tool registration)")
print("   - Agent must know specific tool names")
print("   - Additional layer of indirection")

print("\n🎯 APPROACH 2: Direct Event-Based")
print("=================================")
print("✅ Pros:")
print("   - Direct event emission from agent logic")
print("   - Simpler integration with transport layer")
print("   - More flexible payload structures")
print("   - Can emit multiple events in sequence")
print()
print("❌ Cons:")
print("   - Agent needs custom logic for event handling")
print("   - Less standardized than tool pattern")

print("\n💡 RECOMMENDATION:")
print("==================")
print("🎯 Use TOOL-BASED approach for your API key requests!")
print("   Reasons:")
print("   1. Your request_api_key.py is already well-designed")
print("   2. Tools provide clear interfaces for agents")
print("   3. AG2 handles tool registration automatically") 
print("   4. Easier to test and debug")
print("   5. Follows AG2 best practices")

print("\n✅ Your current request_api_key.py approach is CORRECT!")
print("   Just make sure it's properly registered as a tool with your agents.")

## 6. API Key Request Implementation

Your current `request_api_key.py` is actually well-designed! Let's see how to properly integrate it:

In [ ]:
# How your request_api_key tool should work with AG2

# 1. AGENT CALLS TOOL (Your current approach - this is correct!)
async def mock_request_api_key(service: str, description: str = None) -> Dict[str, Any]:
    """Mock version of your request_api_key function"""
    print(f"🔑 Agent called request_api_key tool for: {service}")
    
    # Emit UI event (your current emit_ui_tool_event)
    event_id = f"api_key_{service}_{str(uuid.uuid4())[:8]}"
    print(f"📡 Emitting UI event: {event_id}")
    
    # Wait for user response (your current wait_for_ui_tool_response) 
    print(f"⏳ Waiting for user to enter {service} API key...")
    await asyncio.sleep(0.1)  # Simulate user interaction
    
    # Mock response from user
    response = {
        "status": "success",
        "api_key": f"sk-mock-{service}-key",
        "service": service
    }
    
    print(f"✅ User provided API key for {service}")
    return response

# 2. CONTEXT VARIABLES STORE THE RESULT
async def api_key_workflow_example():
    """Example of complete API key workflow"""
    
    # Create context variables for storing results
    context_vars = ContextVariables()
    
    # Agent calls the tool
    print("🤖 Agent: I need an OpenAI API key to continue...")
    api_key_response = await mock_request_api_key("openai", "Required for LLM operations")
    
    # Store result in context variables (for other agents to use)
    if api_key_response["status"] == "success":
        # Get existing API keys or create new dict
        api_keys = context_vars.get("api_keys", {})
        api_keys["openai"] = api_key_response["api_key"]
        context_vars.set("api_keys", api_keys)
        
        print(f"💾 Stored API key in context variables")
        print(f"🔑 Available API keys: {list(api_keys.keys())}")
    
    return context_vars

# Run the example
result_context = await api_key_workflow_example()

print("\n🎯 COMPLETE FLOW:")
print("1. Agent calls request_api_key() tool")
print("2. Tool emits UI event to frontend") 
print("3. User enters API key in UI")
print("4. Tool receives response via transport layer")
print("5. Tool returns API key to agent")
print("6. Agent stores API key in context variables")
print("7. Other agents can access API key from context")

print("\n✅ Your request_api_key.py already implements steps 2-5 correctly!")

## 7. Context Variables for State Management

Let's see how to properly use context variables to store and manage application state:

In [ ]:
# Proper Context Variables for Your MozaiksAI Workflow

def create_workflow_context_variables():
    """Create context variables for your Generator workflow"""
    
    context_vars = ContextVariables()
    
    # 1. USER INFORMATION
    context_vars.set("user_id", "user_12345")
    context_vars.set("enterprise_id", "enterprise_abc")
    context_vars.set("chat_id", "chat_67890")
    
    # 2. WORKFLOW STATE
    context_vars.set("workflow_type", "generator")
    context_vars.set("workflow_stage", "initialization")  # initialization, agent_creation, handoffs, completion
    context_vars.set("current_agent", "ContextAgent")
    
    # 3. PROJECT INFORMATION (from user input)
    context_vars.set("concept_overview", "Build a customer support chatbot with AI capabilities")
    context_vars.set("target_audience", "E-commerce businesses")
    context_vars.set("technical_requirements", [
        "React frontend",
        "Node.js backend", 
        "OpenAI integration",
        "Real-time messaging"
    ])
    
    # 4. API KEYS (populated by tools)
    context_vars.set("api_keys", {})  # Will be populated by request_api_key tool
    
    # 5. GENERATED OUTPUTS (populated by agents)
    context_vars.set("generated_agents", [])     # From AgentsAgent
    context_vars.set("generated_handoffs", [])   # From HandoffsAgent  
    context_vars.set("generated_orchestration", {})  # From OrchestratorAgent
    
    return context_vars

# Example of how agents update context variables
def simulate_agent_workflow():
    """Simulate how different agents interact with context variables"""
    
    context = create_workflow_context_variables()
    
    print("🎯 INITIAL CONTEXT STATE:")
    print(f"   Concept: {context.get('concept_overview')}")
    print(f"   Stage: {context.get('workflow_stage')}")
    print(f"   API Keys: {len(context.get('api_keys', {}))}")
    
    # ContextAgent updates analysis results
    context.set("agentic_function_recommendation", "Intelligent ticket routing agent")
    context.set("workflow_stage", "agent_creation")
    context.set("current_agent", "AgentsAgent")
    
    # AgentsAgent generates agent definitions
    generated_agents = [
        {"name": "RouterAgent", "purpose": "Route customer tickets"},
        {"name": "AnalyzerAgent", "purpose": "Analyze ticket sentiment"},
        {"name": "ResponseAgent", "purpose": "Generate responses"}
    ]
    context.set("generated_agents", generated_agents)
    context.set("workflow_stage", "handoffs")
    context.set("current_agent", "HandoffsAgent")
    
    # API key gets added by tool
    api_keys = context.get("api_keys", {})
    api_keys["openai"] = "sk-actual-key-from-user"
    context.set("api_keys", api_keys)
    
    print("\n🔄 UPDATED CONTEXT STATE:")
    print(f"   Recommendation: {context.get('agentic_function_recommendation')}")
    print(f"   Stage: {context.get('workflow_stage')}")
    print(f"   Generated Agents: {len(context.get('generated_agents', []))}")
    print(f"   API Keys: {list(context.get('api_keys', {}).keys())}")
    
    return context

workflow_context = simulate_agent_workflow()

print("\n✅ KEY INSIGHTS:")
print("1. Context variables store workflow state between agents")
print("2. Agents can read AND update context variables")
print("3. Tools (like request_api_key) update context variables")
print("4. Context variables persist throughout the entire workflow")
print("5. All agents automatically receive the current context")

## 8. Event Response Handling Patterns

Finally, let's look at robust patterns for handling asynchronous responses:

In [ ]:
# Robust Event Response Handling (like your request_api_key.py)

class RobustEventHandler:
    """Example of robust event handling with timeouts and error management"""
    
    def __init__(self):
        self.pending_events = {}
        self.event_responses = {}
    
    async def request_user_input(
        self, 
        tool_id: str, 
        payload: Dict[str, Any],
        timeout: float = 300.0,
        retry_count: int = 3
    ) -> Dict[str, Any]:
        """Robust event handling with timeout and retry logic"""
        
        for attempt in range(retry_count):
            try:
                # Generate unique event ID
                event_id = f"{tool_id}_{str(uuid.uuid4())[:8]}"
                
                print(f"🎯 Attempt {attempt + 1}: Requesting user input")
                print(f"   Tool: {tool_id}")
                print(f"   Event ID: {event_id}")
                
                # Emit event to frontend
                await self._emit_event(event_id, tool_id, payload)
                
                # Wait for response with timeout
                response = await self._wait_for_response(event_id, timeout)
                
                # Validate response
                if self._is_valid_response(response):
                    print(f"✅ Successfully received valid response")
                    return response
                else:
                    print(f"❌ Invalid response received, retrying...")
                    continue
                    
            except asyncio.TimeoutError:
                print(f"⏰ Timeout on attempt {attempt + 1}")
                if attempt == retry_count - 1:
                    raise Exception(f"Failed to get response after {retry_count} attempts")
                continue
            except Exception as e:
                print(f"❌ Error on attempt {attempt + 1}: {e}")
                if attempt == retry_count - 1:
                    raise
                continue
        
        raise Exception("All retry attempts failed")
    
    async def _emit_event(self, event_id: str, tool_id: str, payload: Dict[str, Any]):
        """Emit event to transport system"""
        print(f"📡 Emitting event: {event_id}")
        # Your transport.send_tool_event() goes here
        await asyncio.sleep(0.05)  # Simulate network
        
    async def _wait_for_response(self, event_id: str, timeout: float) -> Dict[str, Any]:
        """Wait for response with timeout"""
        print(f"⏳ Waiting for response (timeout: {timeout}s)")
        
        # Simulate variable response times
        await asyncio.sleep(0.2)  # Simulate user thinking time
        
        # Mock response
        return {
            "event_id": event_id,
            "status": "success",
            "data": {"user_input": "User provided data"},
            "timestamp": datetime.utcnow().isoformat()
        }
    
    def _is_valid_response(self, response: Dict[str, Any]) -> bool:
        """Validate response structure"""
        required_fields = ["event_id", "status", "data"]
        return all(field in response for field in required_fields)

# Test robust event handling
async def test_robust_handling():
    handler = RobustEventHandler()
    
    try:
        response = await handler.request_user_input(
            tool_id="api_key_input",
            payload={
                "service": "openai",
                "label": "OpenAI API Key",
                "required": True
            },
            timeout=30.0,
            retry_count=3
        )
        
        print(f"\n🎉 Success! Response: {response['status']}")
        print(f"📊 Data: {response['data']}")
        
    except Exception as e:
        print(f"\n💥 Failed: {e}")

await test_robust_handling()

print("\n🏗️ YOUR REQUEST_API_KEY.PY IMPLEMENTATION:")
print("==========================================")
print("✅ Already includes:")
print("   - Unique event ID generation")
print("   - Timeout handling (300s default)")
print("   - Proper error handling and exceptions")
print("   - Response validation")
print("   - Transport layer integration")
print()
print("💡 Suggested improvements:")
print("   - Add retry logic for failed requests")
print("   - Add response validation for API key format")
print("   - Store successful API keys in context variables")
print("   - Add logging for debugging user interaction issues")

## 🎯 Summary & Recommendations

### Context Variables (✅ Fix Needed)
**What they ARE**: Simple key-value storage for workflow state
**What they're NOT**: Tools or functions that agents call

**Fix for your ContextVariables.py**:
```python
# ❌ WRONG: Don't make tools for context variables
def get_context_variable(name: str):  # This is wrong!
    return context.get(name)

# ✅ RIGHT: Just set and use context variables directly
context_vars = ContextVariables()
context_vars.set("concept_overview", "Build a chatbot")
# Agents automatically receive these values
```

### Event Processing (✅ Already Correct!)
**Your `request_api_key.py` is well-designed!** It correctly:
- Emits UI events to frontend
- Waits for user responses
- Handles timeouts and errors
- Returns structured data

### Recommended Architecture:
1. **Context Variables**: Store workflow state, user info, API keys
2. **Tools**: Interactive functions like `request_api_key()`  
3. **Events**: Asynchronous UI interactions handled by tools

### Action Items:
1. ✅ Keep your `request_api_key.py` tool as-is
2. 🔧 Fix `ContextVariables.py` to just set/get simple values
3. 📝 Make sure agents store API keys in context variables after getting them
4. 🎯 Use context variables for all state sharing between agents